# 第3回 演習課題

## 課題1．単純パーセプトロンの実装と学習

In [1]:
import numpy as np
from sklearn.utils import shuffle
np.random.seed(1234)

1.層をLayerクラスとして定義

In [2]:
class Layer:
    def __init__(self,in_dim,out_dim,function):
        self.W = np.zeros((in_dim,out_dim))
        self.b = np.zeros(out_dim)
        self.function = function

    #forward propagation
    def fprop(self,x):
        u = np.dot(x,self.W)+self.b
        z = self.function(u)
        return z

2.ステップ関数

ヒント：ステップ関数

* $u\geq0$のとき，$f(u)=+1$
* $u<0$のとき，$f(u)=-1$

In [3]:
def step(x):
    y = np.sign(x)
    y[y==0]=1
    return y

4.データセットの設定とレイヤーインスタンス

In [4]:
#OR
train_X = np.array([[0,1],[1,0],[0,0],[1,1]])
train_y = np.array([[1],[1],[-1],[1]])
test_X,test_y = train_X,train_y

layer = Layer(2,1,step)

5.train関数とtest関数

ヒント：パーセプトロン学習則

$y_n\neq d_n$のとき
* $w^{(t+1)}=w^{(t)}-\epsilon x_nd_n$　
* $b^{(t+1)}=b^{(t)}-\epsilon d_n$

In [5]:
def train(x,d,eps=1):
    #forward propagation
    y = layer.fprop(x)

    #update parameters
    if d*y!=1:
        layer.W = layer.W + eps*d*x.T
        layer.b = layer.b + eps*d*np.ones(1)

def test(x):
    y = layer.fprop(x)
    return y

5.パラメータの更新

In [6]:
#epoch
for epoch in range(10):
    #online learning
    for x,y in zip(train_X,train_y):
        train(x[np.newaxis,:],y[np.newaxis,:],eps=1)
pred_y = test(test_X)
print pred_y

[[ 1.]
 [ 1.]
 [-1.]
 [ 1.]]


##課題2．活性化関数とその微分の実装

1.シグモイド関数とその微分

In [7]:
def sigmoid(x):
    return 1/(1+np.exp(-x))
def diff_sigmoid(x):
    return sigmoid(x)*(1-sigmoid(x))

２.ソフトマックス関数とその微分

In [8]:
def softmax(x):
    return np.exp(x)/np.sum(np.exp(x),axis=1)[:,np.newaxis]
def diff_softmax(x):
    return softmax(x)*(np.ones(x.shape)-softmax(x))

2.tanh関数とその微分

In [9]:
def tanh(x):
    return (np.exp(x)-np.exp(-x))/(np.exp(x)+np.exp(-x))
def diff_tanh(x):
    return 1-tanh(x)**2

## 課題3．多層パーセプトロンの実装と学習

1.Layerクラス

In [10]:
class Layer:
    def __init__(self,in_dim,out_dim,function,diff_function):
        #Xavier
        self.W = np.random.uniform(
                                    low=-np.sqrt(6./(in_dim+out_dim)), 
                                    high=np.sqrt(6./(in_dim+out_dim)), 
                                    size=(in_dim, out_dim))
        self.b = np.zeros(out_dim)
        self.function = function
        
        self.diff_function = diff_function
        self.u     = None
        self.delta = None

    #foward propagation
    def fprop(self,x):
        u = np.dot(x,self.W)+self.b
        z = self.function(u)
        self.u = u
        return z

    #back propagation
    def bprop(self,delta,W):
        delta = self.diff_function(self.u)*np.dot(delta,W.T)
        self.delta = delta
        return delta

2.ネットワーク全体の順伝播

In [11]:
def fprops(layers, x):
    z = x
    for layer in layers:
        z = layer.fprop(z)
    return z

3.ネットワーク全体の誤差逆伝播

In [12]:
def bprops(layers, delta):
    for i,layer in enumerate(layers[::-1]):
        if i==0:
            layer.delta = delta
        else:
            delta = layer.bprop(delta,_W)
        _W = layer.W

4.データセットの設定とネットワークの定義

In [13]:
#XOR
train_X = np.array([[0,1],[1,0],[0,0],[1,1]])
train_y = np.array([[1],[1],[0],[0]])
test_X,test_y = train_X,train_y

layers = [Layer(2,3,sigmoid,diff_sigmoid),
          Layer(3,1,sigmoid,diff_sigmoid)]

5.train関数とtest関数

In [14]:
def train(X,d,eps=1):
    #forward propagation
    y = fprops(layers,X)
        
    #cost function & delta
    cost = np.sum(-d*np.log(y)-(1-d)*np.log(1-y))
    delta = y-d
    
    #back propagation
    bprops(layers,delta)

    #update parameters
    z = X
    for layer in layers:
        dW = np.dot(z.T,layer.delta)
        db = np.dot(np.ones(len(z)),layer.delta)

        layer.W = layer.W - eps*dW
        layer.b = layer.b - eps*db

        z = layer.fprop(z)
        
    #train cost
    y = fprops(layers,X)
    cost = np.sum(-d*np.log(y)-(1-d)*np.log(1-y))
    
    return cost

def test(X,d):
    #test cost
    y = fprops(layers,X)
    cost = np.sum(-d*np.log(y)-(1-d)*np.log(1-y))
    return cost,y

6.パラメータの更新

In [15]:
#epoch
for epoch in range(100):
    #online learning
    train_X, train_y = shuffle(train_X, train_y)
    for x,y in zip(train_X,train_y):
        train(x[np.newaxis,:],y[np.newaxis,:])
    cost,pred_y = test(test_X,test_y)
print pred_y

[[ 0.96623963]
 [ 0.96693608]
 [ 0.01315283]
 [ 0.07086153]]


## 宿題．MNISTデータセットを多層パーセプトロンで学習せよ

ヒント
* 出力yはone-of-k表現
* 最終層の活性化関数はsoftmax関数，誤差関数は多クラス交差エントロピー
* 最終層のデルタは教科書参照

In [ ]:
import matplotlib.pyplot as plt
import numpy

from sklearn.utils import shuffle
from sklearn.cross_validation import train_test_split
from sklearn.metrics import f1_score

from sklearn.datasets import fetch_mldata
mnist = fetch_mldata('MNIST original', data_home='.')

X, y = shuffle(mnist.data, mnist.target)
X = X / 255.0
train_X, test_X, train_y, test_y = train_test_split(X, y, test_size=0.2)